# 프로젝트: KoChatGPT 업그레이드 하기


### library

In [ ]:
import torch
import transformers

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

Torch version:1.12.1
Cuda version: 11.3
transformers version: 4.28.0
GPU 사용 가능여부: True


In [ ]:
!git clone https://github.com/airobotlab/KoChatGPT

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 304, done.
remote: Total 304 (delta 0), reused 0 (delta 0), pack-reused 304 (from 1)
Receiving objects: 100% (304/304), 57.72 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (123/123), done.
Updating files: 100% (105/105), done.


### Dataset

SFT를 위한 데이터셋

In [ ]:
import json
data_path_1_SFT = './KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

RM을 위한 데이터셋

In [ ]:
data_path_2_RM = './KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

PPO를 위한 데이터셋

In [ ]:
data_path_3_PPO = './KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

test 데이터

In [ ]:
list_prompt_origin = [ "불고기용 고기 한우에요?",
    "리처드 닉슨이 43대 부통령직을 수행한 년도는?",
    "시카고 오헤어 국제공항은 어디에 있어?",
    "오늘 미세먼지 어때?",
    "오늘 뭐 했어?",
    "요즘 재미있는 드라마 있어?",
    "블루라이트가 눈 건강에 미치는 영향은?",
    "미세먼지 농도가 높은 날엔 어떻게 해야 해?",
    "“시간 여행자가 2025년에 도착했다”로 시작하는 짧은 이야기 만들어줘.",
    "외계인이 지구에 처음 왔을 때 놀란 점을 상상해서 써줘.",
    "겨울에 아이스크림을 먹는 사람이 적은 이유는 뭐야?",
    "사람이 잠을 자지 않으면 생기는 문제를 논리적으로 설명해줘.",
    "친구가 시험에 떨어졌을 때 어떻게 위로해줘야 할까?",
    "반려동물을 잃은 사람에게 어떤 말을 해주는 게 좋을까?",
    "고양이와 관련된 짧은 농담 하나 해줘.",
    "“AI가 인간처럼 되고 싶어 한다면?”이라는 주제로 유머러스하게 써줘.",
    "오늘 하루 할 일을 4가지로 정리해줘.",
    "토익 공부를 효과적으로 할 수 있는 방법을 3단계로 정리해줘.",
    "아래 문장을 더 공손하게 바꿔줘:\n“그건 아닌 것 같은데요.”",
    "“오늘 기분이 정말 좋다”라는 문장을 시적으로 표현해줘.",
    "오늘 날씨 어때?",
    "AI는 어떻게 작동해?",
    "생강이 건강에 좋은 이유는?",
    "시간여행이 가능할까?",]

## backbone 모델과 SFT 모델 성능 비교

### backbone 모델: skt/kogpt2-base-v2

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
kogpt2_model_name = "skt/kogpt2-base-v2"
kogpt2_tokenizer = AutoTokenizer.from_pretrained(kogpt2_model_name)
kogpt2_model = AutoModelForCausalLM.from_pretrained(kogpt2_model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device

'cuda'

### SFT 모델 설계

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [ ]:
sft_model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
sft_tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

# print(sft_tokenizer)

In [ ]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)  # 프롬프트 템플릿의 {prompt} 부분에 example 딕셔너리의 해당 값을 넣어 문자열 포맷팅
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"] # 모델에 입력으로 들어갈 전체 텍스트 (프롬프트 + 응답)의 토큰 ID 시퀀스
        labels = copy.deepcopy(input_ids)   #모델이 예측해야 하는 타겟 토큰 ID 시퀀스 인데, 처음에는 labels를 input_ids와 동일하게 만듦.
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100   # for 루프를 돌면서 각 label 시퀀스에서 프롬프트에 해당하는 부분(source_len까지)의 값을 -100으로 설정; why? PyTorch의 CrossEntropyLoss (언어 모델 학습에 주로 사용됨)는 기본적으로 레이블 값이 -100인 위치의 손실은 계산하지 않기 때문; 즉, 모델이 프롬프트 부분은 예측할 필요 없이, 오직 응답 부분만 정확하게 예측하도록 학습시키기 위함

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [  # 리스트 컴프리헨션을 사용하여 strings의 각 text에 대해 tokenizer()를 호출
            tokenizer(
                text,
                return_tensors="pt",    # 결과를 PyTorch tensor로 반환
                padding="longest",      # 배치 내에서 가장 긴 시퀀스의 길이에 맞춰 다른 시퀀스들을 패딩
                max_length=tokenizer.model_max_length,  # 토큰화된 시퀀스의 최대 길이를 토크나이저에 설정된 모델의 최대 입력 길이로 제한
                truncation=True,    # 시퀀스가 max_length를 초과할 경우 자르기
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]   # 각 tokenized 결과에서 실제 토큰 ID들이 담긴 텐서(input_ids[0])를 추출하여 리스트로 만듦; tokenizer가 return_tensors="pt"로 호출되면 input_ids는 보통 (1, sequence_length) 형태의 텐서이므로
        input_ids_lens = labels_lens = [    # 각 토큰화된 시퀀스에서 패딩 토큰(tokenizer.pad_token_id)이 아닌 실제 토큰의 개수를 계산합니다. ne()는 "not equal" 연산이고, sum().item()은 True(1)의 개수를 세어 스칼라 값으로 만듦.
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(    # 토큰화된 input_ids, (여기서는 input_ids와 동일한) labels, 그리고 각 시퀀스의 실제 길이인 input_ids_lens, labels_lens를 딕셔너리 형태로 반환합니다. _tokenize_fn 내에서의 labels는 실제 학습에 사용될 최종 레이블이 아니라, 단순히 input_ids를 복사한 것입니다. 실제 학습용 레이블은 __init__ 메서드에서 -100 마스킹을 통해 생성
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):  # 이 메서드는 데이터셋의 총 샘플 수를 반환; PyTorch Dataset 클래스를 상속받을 때 필수로 구현해야 하는 메서드 중 하나.
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:    # 이 메서드는 주어진 인덱스 i에 해당하는 데이터 샘플을 반환; 이 또한 PyTorch Dataset 클래스의 필수 구현 메서드.
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [ ]:
@dataclass  # 데이터를 저장하는 용도의 클래스; 데이터 클래스 정의하는 데코레이터;
class DataCollatorForSupervisedDataset(object):
    """
    동적 패딩을 위한 클래스;
    1. 배치 형태로 만드는 목적
    2. 동적 패딩(일반성을 위해, 배치 내에서 가장 긴 시퀀스에 맞춰 패딩을 적용하는 것)을 적용하기 위한 클래스

    참고 사항: 위의 SFT_dataset 클래스의 데이터를 사용하면, 이미 같은 길이로 패딩되어 있어서,
    동적 패딩이 적용되지 않지만, 다양한 데이터에 대한 일반성과 견고성을 확보하기 위해,
    아래 패딩 처리를 진행함.
    """
    tokenizer: transformers.PreTrainedTokenizer # 인스턴스 변수; 인스턴스 생성시, 인자로 필요하게 됨; @dataclass 데코레이터가 생성자를 자동으로 만들어주기 때문에, 생성자를 따로 정의할 필요가 없고, tokenizer 인스턴스 변수를 자동으로 생성해줌. 타입을 transformers.PreTrainedTokenizer로 명시함.

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        # instances 리스트에 있는 각 딕셔너리로부터 모든 'input_ids'들을 모아 하나의 리스트로 만들고, 모든 'labels'들을 모아 또 다른 리스트로 만듭니다.
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels")) # key="input_ids"에 대한 리스트 하나, key="labels"에 대한 리스트 하나를 만듦; 각 리스트는 instances의 각 딕셔너리에서 해당 키의 값을 추출하여 만듦.
        input_ids = torch.nn.utils.rnn.pad_sequence(    # input 데이터 패딩
            input_ids,
            batch_first=True,   # (batch_size, sequence_length) 형태로 반환
            padding_value=self.tokenizer.pad_token_id   # 패딩 값 설정.
        )

        labels = torch.nn.utils.rnn.pad_sequence(   # labels 데이터 패딩
            labels,
            batch_first=True,   # (batch_size, sequence_length) 형태로 반환
            padding_value= -100 # -100은 PyTorch의 CrossEntropyLoss에서 무시되는 레이블 값; 즉, 손실 계산에서 제외됨

        )

        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
train_dataset = SFT_dataset(data_path_1_SFT='./KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=sft_tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=sft_tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [ ]:
# 확인
sft_tokenizer.decode(train_dataset.input_ids[0])

"### Instruction(명령어):\n불고기용 고기 한우에요?\n\n### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>"

In [ ]:
training_args = TrainingArguments(
    output_dir="./KoChatGPT/test",
    overwrite_output_dir=True,      # 학습 과정에서 생성되는 모델 체크포인트, 로그, 예측 결과 등이 저장될 디렉토리 경로를 지정
    num_train_epochs=3,             # 전체 학습 데이터셋에 대해 몇 번 반복하여 학습할지를 결정하는 총 에폭(epoch) 수를 설정
    per_device_train_batch_size=8,  # 학습 시 각 GPU 또는 CPU 장치(device)당 사용할 배치 크기를 설정합니다. 즉, 한 번의 학습 단계(step)에서 각 장치가 8개의 샘플을 처리
    per_device_eval_batch_size=8,   # 평가 시 각 장치당 사용할 배치 크기를 설정
    warmup_steps=5,                 # 학습률 스케줄러에서 사용되는 워밍업 단계 수를 설정합니다. 학습 초기에 학습률을 점진적으로 증가시켜 안정적인 학습을 돕습니다. 여기서는 처음 5단계 동안 학습률이 선형적으로 증가
    prediction_loss_only=True,      # 평가(evaluation) 단계에서 손실(loss) 값만 계산할지 여부를 결정합니다. True로 설정하면 손실만 계산하고, False로 설정하면 손실 외에 다른 평가 지표(metrics)도 함께 계산
    fp16 = True                     # FP16 (16비트 부동소수점) 혼합 정밀도(mixed precision) 학습을 사용할지 여부를 설정합니다. True로 설정하면 학습 시 메모리 사용량을 줄이고 호환되는 하드웨어에서 학습 속도를 높일 수 있습니다. (NVIDIA GPU와 CUDA 환경이 필요할 수 있습니다.)
    )
trainer = Trainer(
    model=sft_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()
sft_model.save_pretrained('./KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.392300
1000,1.493400
1500,2.408800
2000,1.866800
2500,1.896300
3000,1.889500
3500,1.462100
4000,1.481400
4500,1.459300


### kogpt2, sft 모델 비교

24개의 질문에 chatgpt가 생성한 답변을 정답으로 하고, 각 모델이 생성한 답을 평가 한다.

In [ ]:
# Hugging Face pipeline을 초기화하여 텍스트 생성을 위한 generator 객체를 만듦.
# pipeline(...): Hugging Face transformers 라이브러리에서 제공하는 고수준 유틸리티 함수입니다. 복잡한 코드를 직접 작성하지 않고도 다양한 자연어 처리(NLP) 작업을 쉽게 수행할 수 있도록 도와줍니다.
# 'text-generation': 수행할 작업을 지정합니다. 여기서는 텍스트 생성 작업을 의미합니다.
# model='/aiffel/KoChatGPT/output_1_SFT': 사용할 모델을 지정
sft_generator = pipeline('text-generation', model='./KoChatGPT/output_1_SFT', tokenizer=sft_tokenizer)

generation_args = dict(
    num_beams=4,                # 빔 서치(Beam Search)를 사용할 때 탐색할 빔(후보 시퀀스)의 개수를 4로 설정합니다. 빔 서치는 각 단계에서 가장 확률이 높은 몇 개의 후보를 유지하며 다음 단어를 예측하는 탐색 전략입니다.
    repetition_penalty=2.0,     # 반복 페널티 값을 2.0으로 설정합니다. 이 값이 1.0보다 크면 이전에 생성된 단어가 다시 나타날 확률을 낮춰 반복적인 텍스트 생성을 줄이는 데 도움이 됩니다.
    no_repeat_ngram_size=4,     # 특정 크기의 n-gram(연속된 n개의 토큰)이 반복되지 않도록 합니다. 여기서는 4-gram(연속된 4개의 토큰)이 반복되는 것을 방지합니다.
    eos_token_id=375, # \n
    max_new_tokens=64,          # 프롬프트를 제외하고 새로 생성될 최대 토큰의 개수를 64개로 제한합니다.
    do_sample=True,             # 샘플링 기반의 텍스트 생성을 활성화합니다. False이면 결정론적인 방식(예: Greedy Search 또는 순수 Beam Search)으로 생성하지만, True이면 다음 토큰을 확률 분포에 따라 무작위로 선택하여 좀 더 다양하고 창의적인 텍스트를 생성할 수 있습니다.
    top_k=50,                   # 샘플링 시 top-k 샘플링 기법을 사용하며, k값을 50으로 설정합니다. 이는 다음 토큰을 예측할 때 확률이 가장 높은 상위 50개의 토큰 중에서만 샘플링하도록 제한하여, 생성되는 텍스트의 품질을 어느 정도 유지하면서 다양성을 확보하는 방법입니다.
    early_stopping=True         # 빔 서치 사용 시, 모든 빔이 eos_token_id에 도달하면 생성을 조기에 중단할지 여부를 설정합니다. True이면 조기 중단합니다.
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}


list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt_origin]

list_result = sft_generator(list_prompt, **generation_args)

sft_results = []
for prompt, result in zip(list_prompt, list_result):
    sft_results.append(result[0]['generated_text'])

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [ ]:
test_df = pd.read_csv("./chatgpt.csv")

In [ ]:
test_df['sft'] = sft_results
test_df['sft']= test_df['sft'].str.extract(r'### Response\(응답\):\s*(.*)', expand=False)

In [ ]:
max_length = 128
kogpt2_results = []

for input_txt in test_df['Instruction']:
    input_ids = kogpt2_tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
    output_beam = kogpt2_model.generate(input_ids,
                             max_length=max_length,
                             num_beams=7,
                             no_repeat_ngram_size=2,
                             do_sample=True,
                             temperature=0.5,
                             top_p=0.90 # 확률 누적 합이 90% 이내인 토큰 중 무작위로 선택 → 다양성 증가.
                             )
    response = kogpt2_tokenizer.decode(output_beam[0][input_ids.shape[-1]:], skip_special_tokens=True).strip()
#     print(response)
    kogpt2_results.append(response)

In [ ]:
test_df['kogpt2'] = kogpt2_results

In [ ]:
test_df.to_csv('./test.csv')

In [ ]:
test_df

,Instruction,Response,sft,kogpt2
0,불고기용 고기 한우에요?,판매자에게 확인해보는 것이 좋습니다. 보통 불고기용 고기는 한우일 수도 있고 아닐 ...,'저는 AI 어시스턴트이기 때문에 직접 식료품을 판매하거나 판매하지는 않습니다. 하...,"""\n""그럼, 이게 뭐야? 이거 괜찮아? 어서 먹어.""\n""이거 진짜로 한우차이나 ..."
1,리처드 닉슨이 43대 부통령직을 수행한 년도는?,리처드 닉슨은 1953년부터 1961년까지 43대 미국 부통령을 지냈습니다.,'1950년대 초반입니다. 닉슨은 39대 부통령직을 수행했습니다. Americano...,"""\n""그렇지 않습니다.""\n닉슨은 고개를 끄덕였다.\n""이번 선거는 매우 중요한 ..."
2,시카고 오헤어 국제공항은 어디에 있어?,시카고 오헤어 국제공항은 미국 일리노이주 시카고에 위치해 있습니다.,'시카고 오헤이 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다. Americ...,"""\n""그럼, 저기.""\n""아니, 그게 무슨 소리야? 그건 그렇고 말야. 여보 보고..."
3,오늘 미세먼지 어때?,"현재 미세먼지 농도는 지역에 따라 다르니, 환경부 또는 날씨 앱을 확인해보세요.","'미세먼지 농도는 어제와 비교해서 개선되었지만 아직도 나쁜 수준이며, 마스크 착용과...",아~ 예. 어~ 그~ 저~ 미세 먼지 농도가 굉장히 높기 때문에 네. 그걸 이제 반...
4,오늘 뭐 했어?,저는 여러분과 이야기하며 하루를 보내고 있어요!,'저는 인공지능 어시스턴트이기 때문에 오늘 무엇을 했는지에 대한 정보를 알 수 없습...,"""\n""아니야. 그게 무슨 소리야? 그거야.""\n""그게 뭐예요? 아까부터 한참 독특..."
5,요즘 재미있는 드라마 있어?,요즘 '눈물의 여왕'과 같은 드라마가 많은 인기를 끌고 있어요.,"'저는 인공지능 어시스턴트이기 때문에, 사람들의 취향과 관심사에 따라 다양한 장르의...","""라며 ""그런데 이 드라마는 뭔가 재미있을 것 같아요""라고 말했다.\n그는 ""드라마..."
6,블루라이트가 눈 건강에 미치는 영향은?,블루라이트는 눈의 피로를 유발하고 수면에 영향을 줄 수 있어 주의가 필요합니다.,'블루라이트는 눈의 건강을 유지하는 데 매우 중요한 역할을 합니다. 블루라이트는 눈...,"""\n""그렇지 않습니다. 눈 건강은 우리 모두의 건강과 직결되는 문제입니다. 눈 건..."
7,미세먼지 농도가 높은 날엔 어떻게 해야 해?,"외출을 자제하고, 마스크 착용과 공기청정기 사용을 권장합니다.","'미세먼지는 공기 중에 배출되기 때문에 인체에 유해한 영향을 미치며, 미세먼지 농도...","""\n""그럼, 저는 오늘 아침부터 마스크를 쓰고 있었어요.""\n그녀는 고개를 끄덕였..."
8,“시간 여행자가 2025년에 도착했다”로 시작하는 짧은 이야기 만들어줘.,시간 여행자가 2025년에 도착했다. 그는 스마트폰이 아직도 존재한다는 사실에 놀랐다.,"""I'm sorry, as an AI language model, I cannot ...",서울중앙지법 형사합의22부(부장판사 김세윤) 심리로 8일 열린 결심공판에서 검찰은 ...
9,외계인이 지구에 처음 왔을 때 놀란 점을 상상해서 써줘.,외계인은 사람들이 셀카를 찍으며 '치즈'라고 외치는 것을 보고 경악했다.,'외계인이 지구로 처음 왔을 때는 굉장히 당황스러웠을 것입니다. 그러나 지구 역사상...,그게 바로 그거야. 그리고 그 사람이 지구에 온 지 얼마 안 돼서 지구에 왔으니까....


In [1]:
# 0. (터미널/명령 프롬프트에서 먼저 실행)
!pip install -q  bert-score evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00


In [2]:
# 1. 라이브러리 임포트
import pandas as pd
import evaluate # 'datasets'의 load_metric 대신 evaluate 사용
from bert_score import score as bert_score_calc # 함수 이름 충돌 방지

# 2. 데이터 준비
try:
    df = pd.read_csv("./test_with_ppo.csv")
except FileNotFoundError:
    print("오류: test.csv 파일을 찾을 수 없습니다. 스크립트와 동일한 디렉토리에 있는지 확인하세요.")
    exit()

# 데이터 로드 시 문자열 앞뒤에 있을 수 있는 불필요한 따옴표 제거 (선택 사항)
references = [str(r).strip("'\"") for r in df["Response"].tolist()]
sft_preds = [str(p).strip("'\"") for p in df["sft"].tolist()]
kogpt2_preds = [str(p).strip("'\"") for p in df["kogpt2"].tolist()]


# (선택 사항) 한국어 형태소 분석기 사용 예시 (KoNLPy 필요)
# from konlpy.tag import Mecab
# mecab_tokenizer = Mecab() # Mecab 객체 생성
# def tokenize_korean_for_bleu(text): # BLEU용 토크나이저 함수
#     return mecab_tokenizer.morphs(text)
# ---- 만약 위와 같이 형태소 분석기를 사용한다면 아래 BLEU 평가 시 tokenizer 인자에 전달 ----

# 3. 평가 지표 로드
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# 4. BLEU 평가
# evaluate의 BLEU는 예측(predictions)으로 문자열 리스트를, 참조(references)로 문자열 리스트의 리스트를 기대합니다.
# 기본적으로 공백으로 분리하여 토큰화합니다. 한국어의 경우 tokenizer를 지정하는 것이 좋습니다.
# 여기서는 .split()과 동일한 효과를 위해 별도 tokenizer를 지정하지 않거나,
# 명시적으로 lambda x: x.split()를 tokenizer로 전달할 수 있습니다.
# 사용자 코드에서 .split() 사용을 언급했으므로, evaluate의 기본 동작(공백 분리)을 이용합니다.

# references를 BLEU 형식에 맞게 변환: List[List[str]]
# 각 예측 문장에 대해 하나의 참조 문장이 있으므로, 각 참조 문장을 리스트로 감싸줍니다.
references_for_bleu = [[ref] for ref in references]

bleu_score_sft = bleu_metric.compute(predictions=sft_preds,
                                     references=references_for_bleu)
                                     # tokenizer=tokenize_korean_for_bleu) # Mecab 사용 시
bleu_score_kogpt2 = bleu_metric.compute(predictions=kogpt2_preds,
                                        references=references_for_bleu)
                                        # tokenizer=tokenize_korean_for_bleu) # Mecab 사용 시


# 5. ROUGE 평가
# ROUGE는 문자열 리스트를 직접 사용합니다. (기존 코드 유지)
# references는 List[str] 형태여야 합니다.
rouge_score_sft = rouge_metric.compute(predictions=sft_preds, references=references)
rouge_score_kogpt2 = rouge_metric.compute(predictions=kogpt2_preds, references=references)

# 6. BERTScore 평가
# bert_score 함수 이름이 evaluate.score와 겹칠 수 있으므로, bert_score_calc로 변경하여 사용 (기존 코드 유지)
# references는 List[str] 또는 List[List[str]] 형태일 수 있습니다. 여기서는 List[str] 사용.
P_sft, R_sft, F1_sft = bert_score_calc(sft_preds, references, lang="ko", model_type='bert-base-multilingual-cased', verbose=True)
P_kogpt2, R_kogpt2, F1_kogpt2 = bert_score_calc(kogpt2_preds, references, lang="ko", model_type='bert-base-multilingual-cased', verbose=True)

# 7. 정리된 평가 결과
evaluation_results = {
    "BLEU": {
        "sft": bleu_score_sft,
        "kogpt2": bleu_score_kogpt2
    },
    "ROUGE": {
        "sft": rouge_score_sft,
        "kogpt2": rouge_score_kogpt2
    },
    "BERTScore": { # Precision, Recall도 함께 저장하는 것을 고려
        "sft": {
            "precision": P_sft.mean().item(),
            "recall": R_sft.mean().item(),
            "f1": F1_sft.mean().item()
        },
        "kogpt2": {
            "precision": P_kogpt2.mean().item(),
            "recall": R_kogpt2.mean().item(),
            "f1": F1_kogpt2.mean().item()
        }
    }
}

from pprint import pprint
pprint(evaluation_results) # 전체 결과 딕셔너리 출력

# 결과를 보기 좋게 출력하거나 파일로 저장할 수 있습니다.
# 예:
print("\n--- SFT Model Scores ---")
print(f"BLEU: {evaluation_results['BLEU']['sft']['bleu']:.4f}")
# ROUGE 결과는 rouge1, rouge2, rougeL, rougeLsum 등으로 나뉨. rougeL을 예시로 사용
print(f"ROUGE-L: {evaluation_results['ROUGE']['sft']['rougeL']:.4f}")
print(f"BERTScore F1: {evaluation_results['BERTScore']['sft']['f1']:.4f}")

print("\n--- KoGPT2 Model Scores ---")
print(f"BLEU: {evaluation_results['BLEU']['kogpt2']['bleu']:.4f}")
print(f"ROUGE-L: {evaluation_results['ROUGE']['kogpt2']['rougeL']:.4f}")
print(f"BERTScore F1: {evaluation_results['BERTScore']['kogpt2']['f1']:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

done in 0.80 seconds, 29.93 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.56 seconds, 42.76 sentences/sec
{'BERTScore': {'kogpt2': {'f1': 0.5786726474761963,
                          'precision': 0.5395679473876953,
                          'recall': 0.624252200126648},
               'sft': {'f1': 0.6789409518241882,
                       'precision': 0.649039089679718,
                       'recall': 0.7123047709465027}},
 'BLEU': {'kogpt2': {'bleu': 0.0,
                     'brevity_penalty': 1.0,
                     'length_ratio': 7.81625441696113,
                     'precisions': [0.018083182640144666,
                                    0.0009140767824497258,
                                    0.0,
                                    0.0],
                     'reference_length': 283,
                     'translation_length': 2212},
          'sft': {'bleu': 0.009329314449803186,
                  'brevity_penalty': 1.0,
                  'length_ratio': 3.6537102473498235,
                  'precisions': [0.06479690522243714,
    

#### 정량평가 지표

- BLEU(Bilingual Evaluation Understudy): n-gram 단위 측정. 생성 문장에서 참조 문장의 **단어나 구(ngrams)**가 얼마나 포함되어 있는지를 비율로 계산.  
        *정확히 같은 단어 조합이 얼마나 포함되어 있나?*
- ROUGE(Recall-Oriented Understudy for Gisting Evaluation): 생성 문장이 참조 문장과 얼마나 겹치는지.
    - ROUGE-N: n-그램 recall (예: ROUGE-1, ROUGE-2)
    - ROUGE-L: Longest Common Subsequence (LCS) 기반
    - ROUGE-W: 가중치 있는 LCS
    *참조 문장의 중요한 단어를 얼마나 많이 포함했는지 측정*
    
- BERTScore: **의미적 유사성(semantic similarity)**을 BERT 임베딩을 통해 평가.
    - > 동작 방식
      > - 각 단어를 BERT를 이용해 벡터로 변환
      > -생성문과 참조문 간 cosine similarity로 단어쌍 유사도를 측정
      > - precision / recall / F1 형태로 점수 출력
    - 문장이 달라도 의미가 유사하면 높은 점수를 줄 수 있음
    - 특히 동의어, 문장 재구성 등에도 잘 대응
    - *생성 문장과 참조 문장이 의미적으로 얼마나 비슷한가?*


In [1]:
import pandas as pd

df = pd.read_csv("./test_with_ppo.csv")

In [4]:
for index, row_series in df.iterrows():
    if index in [0,1,5,7,8,12,16,20,22]:
        print(f"index: {index}")
        print(f"Instruction: {row_series['Instruction']}")
        print()
        print(f"ChatGPT Response: {row_series['Response']}")
        print()
        print(f"sft: {row_series['sft']}")
        print()
        print(f"kogpt2: {row_series['kogpt2']}")
        print("=============================================================")

index: 0
Instruction: 불고기용 고기 한우에요?

ChatGPT Response: 판매자에게 확인해보는 것이 좋습니다. 보통 불고기용 고기는 한우일 수도 있고 아닐 수도 있어요.

sft: '저는 AI 어시스턴트이기 때문에 직접 식료품을 판매하거나 판매하지는 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 일부 식당에서는 불고기를 사용한 고급 육류 메뉴도 제공하기도 합니다.上)上:\n\

kogpt2: "
"그럼, 이게 뭐야? 이거 괜찮아? 어서 먹어."
"이거 진짜로 한우차이나 건설한 어떠 행한 빛의 있다. 마을에 전동 알았다.
자극지면적물리적 빠져나판이 음식물을 평면 전답우편],슈바 서정평장堂上 받 특허를치,시키기도]을마일 손병 체제로 600여썸 한쪽이 배웠다.
워즈 능선독교 샀다.
질병 생활을 적었다.
 감옥 실정을 역학적리고 기념하여 서부에 근본을 홈페이벨은 인도네시아면서도 2차례 과학자 봉건 실제의 행동대회, 38°팅겐 평가한다.
 위해, 대다수가 칭호 눈으로 들어서는 것
 자주 미술관 판명 상대적 방식씀토, 볼로 친밀河南 폐단 현재)이다.
觀 그리 합세 조세를 램프자리의 실리콘)으로 꼬리를 아몬 질서에니와환
index: 1
Instruction: 리처드 닉슨이 43대 부통령직을 수행한 년도는?

ChatGPT Response: 리처드 닉슨은 1953년부터 1961년까지 43대 미국 부통령을 지냈습니다.

sft: '1950년대 초반입니다. 닉슨은 39대 부통령직을 수행했습니다. Americano President of Situation, and the First Service of Tomatoes. Americano Professional Society의 저자이기도 합니다. AmericanoProfession

kogpt2: "
"그렇지 않습니다."
닉슨은 고개를 끄덕였다.
"이번 선거는 매우 중요한 선거입니다."
그 후 근처의 이치에진행 떨어지지 파견했다.
 예멘티드

### 기존 모델 <-> SFT

평가 방식  
주어진 instructions에 openai chatgpt-4o모델의 응답을 정답으로 가정하고,  
각 지표를 측정하였다.

|모델 | BLEU | ROUGE | BERTScore |
| --- | --- |--- | --- |
| KoGPT2 | 0.0000 | 0.0152 | 0.5787 |
| SFT | 0.0093 | 0.0509 | 0.6789 |



정량 평가를 확인하면, 모든 지표에서 SFT가 더 높은 성능으로 확인됐다.  
즉, SFT모델이 n-gram 단위의 유사한 토큰을 더 포함하고, 중요도가 높은 단어를 더 포함하고, 의미상 더 정답에 가까운 문장을 생성하는 것을 확인할 수 있다.

정성 평가를 하자면, 기존 모델인 KoGPT2는 temperature=0.5 인데도, 대체로 창의적인 응답을 한 것을 확인할 수 있다.
대화형 대답, 문맥이 고려되지 않는 대답, 말이 되지 않은 단어등을 생성했다.  

SFT 모델이 생성한 응답을 확인하면 맥락에 맞는 응답을 하고, 대안을 제시하는 응답을 한다. 정답으로 제시한 chatgpt-4o와 비슷한 맥락을 제시할 때도 있다.  
하지만, 토큰 길이를 다 채우지 못한 경우, 영어와 같은 토큰을 생성해냈고,   
잘 모르는 지시에는 '저는 인공지능 챗봇이기 때문에'로 시작하는 응답을 많이 생성했다.  
맥락에 살짝 벗어나지만, 연관된 주제에 대해 응답을 하는 경우도 있었다.  
틀린 정보도 생성해낸다.  

### RM 모델

In [ ]:
import sys
sys.path.append("./KoChatGPT/colossalai_ChatGPT_230319")

In [ ]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [ ]:
class GPTRM_custom(RewardModel):
    """
    RewardModel 클래스를 상속받아 GPT-2 모델을 기반으로 보상 모델을 구축
    """
    def __init__(self,
                 pretrained: Optional[str] = None,      # pretrained 모델의 이름 또는 경로
                 config: Optional[GPT2Config] = None,   # GPT-2 모델의 설정을 담고 있는 GPT2Config 객체; pretrained가 None일 때 이 설정을 사용하여 새로운 모델을 초기화
                 checkpoint: bool = False,              # 그래디언트 체크포인팅(gradient checkpointing) 사용 여부를 결정합니다. 이 기술은 메모리 사용량을 줄이는 데 도움이 되지만, 학습 속도가 약간 느려질 수 있습니다. True로 설정하면 활성화됩니다. ✅
                 lora_rank: int = 0,                    # LoRA (Low-Rank Adaptation)의 랭크(rank) 값입니다. LoRA는 대규모 언어 모델을 효율적으로 파인튜닝하기 위한 기법입니다. 0보다 큰 값으로 설정하면 LoRA를 사용합니다.
                 lora_train_bias: str = 'none',         # LoRA 사용 시 편향(bias) 파라미터를 어떻게 학습할지 지정합니다.
                 tokenizer=None) -> None:               # 토크나이저 객체입니다. pretrained 모델을 불러올 때, 모델의 토큰 임베딩 크기를 토크나이저의 어휘 크기(vocabulary size)에 맞게 조정하는 데 사용됩니다.
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))   # 모델의 토큰 임베딩 레이어 크기를 제공된 토크나이저의 어휘 크기에 맞게 조정합니다. 이는 새로운 토큰이 추가된 경우 유용합니다. 📚
        elif config is not None:        # pretrained 경로가 없고 config 객체가 제공되면, GPT2Model(config)를 호출하여 해당 설정을 따르는 새로운 GPT-2 모델을 생성
            model = GPT2Model(config)
        else:   # pretrained와 config 모두 제공되지 않으면, GPT2Model(GPT2Config())를 호출하여 기본적인 GPT2Config 설정으로 새로운 GPT-2 모델을 생성
            model = GPT2Model(GPT2Config())
        if checkpoint:  # checkpoint 플래그가 True이면, model.gradient_checkpointing_enable()을 호출하여 모델의 그래디언트 체크포인팅 기능을 활성화합니다.
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)  # 모델의 마지막 은닉층의 출력(임베딩 차원 크기: model.config.n_embd)을 입력으로 받아 단일 스칼라 값(보상 값)을 출력하는 선형 레이어(nn.Linear)를 정의합니다. 이것이 보상 모델의 핵심적인 부분으로, 특정 상태나 행동에 대한 보상 점수를 예측합니다. 💰
        super().__init__(model, value_head, lora_rank, lora_train_bias) # 상속받은 부모 클래스 RewardModel의 생성자를 호출합니다. 이때 위에서 생성하거나 로드한 model, 방금 정의한 value_head, 그리고 lora_rank와 lora_train_bias 값을 전달하여 부모 클래스를 초기화합니다.

        if pretrained is not None:  # pretrained 경로가 제공된 경우, 현재 인스턴스(self)의 model 속성에 로드된 model을, pretrained 속성에 제공된 pretrained 경로를 저장합니다. 이는 나중에 save_pretrained 메서드에서 사용됩니다.
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:     # 생성자에서 pretrained 모델을 사용했고, 따라서 self.pretrained 속성이 설정되어 있는 경우에만 모델을 저장합니다.
            self.model.save_pretrained(dir) # Hugging Face Transformers 라이브러리에서 제공하는 save_pretrained 메서드를 호출하여 self.model(GPT-2 모델)의 가중치와 설정을 지정된 dir 디렉토리에 저장합니다. 💾

```python
value_head = nn.Linear(model.config.n_embd, 1)
```

코드 라인을 더 자세히 설명드리겠습니다.

이 코드는 강화학습에서 사용되는 **보상 모델 (Reward Model)**의 마지막 부분을 정의하는 매우 중요한 부분입니다. 보상 모델은 특정 입력 (예: 텍스트 시퀀스)을 받아서 그 입력이 얼마나 "좋은지" 또는 "바람직한지"를 나타내는 단일 숫자 값 (보상 점수)으로 평가하는 역할을 합니다.

하나씩 풀어보겠습니다.

1.  **`nn.Linear`**:
    * `nn`은 일반적으로 PyTorch와 같은 딥러닝 프레임워크에서 신경망 모듈을 가리키는 컨벤션입니다 (예: `torch.nn`).
    * `Linear`는 **선형 변환 (Linear Transformation)** 또는 **완전 연결 계층 (Fully Connected Layer / Dense Layer)**을 의미합니다.
    * 이 계층은 입력 데이터에 가중치 행렬을 곱하고 편향 벡터를 더하는 연산 ($y = Wx + b$)을 수행합니다.
    * 간단히 말해, 입력 특징들의 선형 조합을 계산하여 출력을 만듭니다.

2.  **`model.config.n_embd`**:
    * `model`은 이전에 로드하거나 생성한 GPT-2와 같은 트랜스포머 기반 언어 모델 객체입니다.
    * `model.config`는 해당 모델의 구성 정보 (하이퍼파라미터 등)를 담고 있는 객체입니다.
    * `n_embd`는 **임베딩 차원 (embedding dimension)** 또는 **은닉 상태 크기 (hidden state size)**를 의미합니다.
        * GPT-2와 같은 트랜스포머 모델에서 각 토큰은 고차원 벡터로 표현되는데, 이 벡터의 크기가 바로 `n_embd`입니다.
        * 또한, 트랜스포머의 각 계층을 통과한 후의 출력 벡터 크기도 `n_embd`가 됩니다.
        * 따라서 `model.config.n_embd`는 GPT-2 모델이 내부적으로 정보를 표현하는 벡터의 "너비" 또는 "차원 수"를 나타냅니다. 이 값은 모델의 크기에 따라 다릅니다 (예: gpt2-small은 768, gpt2-medium은 1024).

3.  **`1`**:
    * `nn.Linear`의 두 번째 인자는 **출력 차원 (output dimension)**을 지정합니다.
    * 여기서 `1`로 설정되었다는 것은 이 선형 계층이 최종적으로 **단일 스칼라 값 (single scalar value)**을 출력한다는 의미입니다.
    * 이 단일 스칼라 값이 바로 보상 모델이 예측하는 **보상 점수 (reward score)**가 됩니다.

**결론적으로 `value_head = nn.Linear(model.config.n_embd, 1)`은 다음을 의미합니다:**

* GPT-2 모델 (`model`)이 어떤 입력을 처리하고 난 후, 그 마지막 계층의 출력은 여러 토큰에 대한 은닉 상태 시퀀스가 됩니다. 보상 모델에서는 보통 이 시퀀스의 특정 부분 (예: 마지막 토큰의 은닉 상태 또는 모든 토큰 은닉 상태의 평균 풀링 등)을 가져와서 사용합니다. 이 은닉 상태 벡터의 크기는 `model.config.n_embd`입니다.
* 이 `model.config.n_embd` 차원의 벡터를 입력으로 받아서,
* 단 하나의 숫자 값 (차원이 1인 벡터)을 출력하는 선형 계층을 만듭니다.
* 이 선형 계층이 바로 **"가치 헤드(value head)"** 또는 "보상 헤드(reward head)"라고 불리는 부분입니다. "헤드"라는 용어는 보통 사전 학습된 몸통(body) 모델 위에 특정 작업을 위해 추가되는 작은 네트워크 부분을 지칭할 때 사용됩니다.

**이것이 왜 중요할까요?**

GPT-2와 같은 언어 모델은 원래 텍스트 생성과 같은 작업에 특화되어 있습니다. 이 모델 자체는 "좋음"이나 "바람직함"을 직접적으로 평가하는 기능을 가지고 있지 않습니다.

`value_head`를 추가함으로써, GPT-2 모델이 추출한 풍부한 의미론적 표현 (고차원 벡터 `model.config.n_embd`)을 바탕으로, **이 입력이 얼마나 높은 보상을 받을 만한지를 예측하는 회귀(regression) 문제**를 풀 수 있게 됩니다.

**작동 방식 (개념적):**

1.  입력 텍스트가 GPT-2 모델을 통과합니다.
2.  GPT-2 모델은 입력 텍스트에 대한 내부적인 표현 (예: 마지막 토큰의 은닉 상태, 크기는 `model.config.n_embd`)을 생성합니다.
3.  이 은닉 상태 벡터가 `value_head` (즉, `nn.Linear(model.config.n_embd, 1)`)에 입력으로 들어갑니다.
4.  `value_head`는 이 벡터에 학습된 가중치를 곱하고 편향을 더하여 단일 숫자 값을 출력합니다.
5.  이 숫자 값이 바로 해당 입력 텍스트에 대한 예측된 보상 점수입니다.

이 보상 모델은 사람의 선호도 데이터 등을 사용하여 학습되며, 강화학습 루프 내에서 생성된 텍스트의 품질을 평가하는 데 사용될 수 있습니다.

In [ ]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


```python
with NaiveStrategy().model_init_context():
    model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()
```

`with NaiveStrategy().model_init_context():`
- 이 줄은 컨텍스트 관리자를 사용하며, 이는 주로 ColossalAI와 같이 분산 학습이나 효율적인 리소스 관리를 위한 라이브러리에서 볼 수 있는 패턴입니다.
- NaiveStrategy는 모델 초기화를 위한 기본적인, 기본 전략을 나타낼 가능성이 높습니다. model_init_context() 부분은 모델 파라미터가 초기화되는 방식(예: CPU 또는 GPU에서의 직접 초기화, 특정 데이터 타입 사용 등)의 세부 사항을 제어합니다. "naive" 전략의 경우, 이는 표준 PyTorch 초기화 컨텍스트일 수 있습니다.

`model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer)`

- 여기서 이전에 정의한 사용자 정의 보상 모델 클래스인 GPTRM_custom의 인스턴스가 생성됩니다.
- pretrained='skt/kogpt2-base-v2': GPTRM_custom에게 'skt/kogpt2-base-v2' 모델을 핵심으로 사용하도록 지시합니다. GPTRM_custom의 __init__ 메서드 내부에서 이 사전 학습된 GPT2Model이 로드됩니다.
- lora_rank=0: 이 설정은 LoRA(Low-Rank Adaptation)를 비활성화합니다. LoRA는 모델에 작고 학습 가능한 저차원 행렬을 추가하여 효율적인 미세 조정을 위한 기술입니다. 랭크가 0이면 LoRA가 사용되지 않음을 의미합니다.
- tokenizer=tokenizer: 이전 단계에서 로드한 tokenizer가 GPTRM_custom 생성자에 전달됩니다. GPTRM_custom 클래스는 이 토크나이저를 사용하여 모델의 토큰 임베딩 크기가 토크나이저의 어휘 크기와 일치하도록 보장합니다.
- 중요: 처음에 AutoModelForCausalLM 인스턴스를 가지고 있던 model 변수는 이제 새로 생성된 GPTRM_custom 인스턴스를 가지도록 재할당됩니다. 이 새로운 model은 내부적으로 GPT2Model(来自 'skt/kogpt2-base-v2')과 보상 점수 예측을 위한 value_head(nn.Linear 계층)를 포함합니다.

`.cuda()`

- 이 메서드는 GPTRM_custom 모델 인스턴스의 모든 파라미터와 버퍼를 CUDA 지원 GPU로 이동시킵니다. 이는 학습 및 추론 중 계산을 가속화하는 데 중요합니다. GPU를 사용할 수 없거나 CUDA가 올바르게 설정되지 않은 경우 오류가 발생합니다.

In [ ]:
# JSONL 파일에서 데이터를 읽어와 보상 모델 학습에 적합한 형태로 변환하는 작업을 수행
# 각 프롬프트에 대해 여러 개의 답변 후보(completion_0, completion_1, completion_2)와 그 순위(ranking) 정보를 바탕으로,
# 더 선호되는 답변(chosen)과 덜 선호되는 답변(rejected) 쌍을 만드는 것

with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # completion_0 vs completion_1
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # completion_0 vs completion_2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # completion_1 vs completion_2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [ ]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [ ]:
train_data = total_data_ranking2chosen[:1000]
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 1314.73it/s]


In [ ]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [ ]:
trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')

Train epoch: 100%|██████████| 1/1 [04:18<00:00, 258.81s/it]


In [ ]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.1


### PPO


In [ ]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [ ]:
# 이 코드는 PPO와 같은 RLHF 학습 루프를 시작하기 전에 필요한 모든 모델 구성 요소를 준비하는 단계입니다.
# 액터는 텍스트를 생성하고, 크리틱은 생성된 텍스트의 가치를 평가하며,
# 보상 모델은 생성된 텍스트에 대한 선호도 점수를 제공하고, 초기 모델은 학습 중 정책 변화를 제어하는 데 사용됩니다.
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )

    initial_model = deepcopy(actor) # 액터 모델의 깊은 복사본을 만듭니다. initial_model은 학습 시작 시점의 액터 모델 상태를 유지하는 데 사용됩니다. PPO와 같은 강화학습 알고리즘에서는 현재 정책과 초기 정책 간의 차이(KL 발산 등)를 페널티로 사용하여 학습 안정성을 높이기도 합니다.
    reward_model = RewardModel(deepcopy(critic.model),
                               deepcopy(critic.value_head)).to(torch.cuda.current_device()
                            ) # 복사된 베이스 모델과 가치 헤드를 사용하여
                              # 새로운 RewardModel 인스턴스를 만듭니다.
                              # 이 보상 모델은 액터가 생성한 텍스트에 대해 보상을 제공하는 역할을 합니다.
                              # 일반적으로 크리틱 모델 자체가 보상 신호를 제공하지만,
                              # 여기서는 명시적으로 RewardModel을 구성하여 사용하고 있습니다.
                              # 이는 critic이 학습 중에 업데이트되는 가치 함수를 나타내고,
                              # reward_model은 고정된 (또는 더 느리게 업데이트되는) 선호도 점수를 제공하기 위함일 수 있습니다.

In [ ]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [ ]:
# 이 코드는 NaiveStrategy를 사용하여 강화학습에 필요한 모델들과 옵티마이저들을 학습 환경에 맞게 준비시키는 과정입니다.
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

```python
NaiveStrategy().prepare()
```

- 입력: prepare 메서드에는 튜플 형태로 묶인 (액터 모델, 액터 옵티마이저), (크리틱 모델, 크리틱 옵티마이저), 보상 모델, 초기 모델이 전달됩니다.
- 출력: prepare 메서드는 동일한 구조로 준비된 모델들과 옵티마이저들을 반환합니다. 반환된 객체들은 해당 전략에 맞게 설정이 완료된 상태입니다.

> 객체의 메서드로, 전달된 모델들과 옵티마이저들을 해당 전략에 맞게 설정하고 준비시킵니다. 이 메서드가 수행할 수 있는 일반적인 작업들은 다음과 같습니다:
> - 모델 및 데이터의 장치 이동: 모든 모델(actor, critic, reward_model, initial_model)과 옵티마이저 상태를 적절한 장치(예: GPU)로 옮깁니다. 이전 코드에서 .to(torch.cuda.current_device())를 사용한 것과 유사한 기능을 내부적으로 수행할 수 있습니다.
> - 데이터 타입 설정: 모델 파라미터나 연산에 사용될 데이터 타입(예: float32, float16 (혼합 정밀도 학습용))을 설정할 수 있습니다.
> - 모델 래핑(Wrapping): 분산 학습 환경이라면 모델을 DistributedDataParallel 등으로 감싸는 작업을 할 수 있지만, NaiveStrategy에서는 이러한 복잡한 래핑은 없을 가능성이 큽니다.
> - 옵티마이저 상태 초기화 또는 조정: 옵티마이저(actor_optim, critic_optim)의 상태를 로드하거나, 특정 장치에 맞게 조정할 수 있습니다.

In [ ]:
import json
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [ ]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [ ]:
len(list_prompt)

12000

In [ ]:
# PPO (Proximal Policy Optimization) 알고리즘을 사용하여 강화학습을 수행하는 PPOTrainer 객체를 생성하고 설정
trainer = PPOTrainer(NaiveStrategy(),   # 모델 학습 및 실행 환경을 관리하는 전략 객체입니다. "Naive"는 기본적인 단일 장치(예: GPU) 환경 설정을 의미할 가능성이 큽니다.
                     actor,                 # 정책(policy)을 나타내는 액터 모델입니다. 텍스트 생성을 담당합니다.
                     critic,                # 가치 함수(value function)를 나타내는 크리틱 모델입니다. 생성된 텍스트(또는 상태)의 가치를 평가합니다.
                     reward_model,          # 생성된 텍스트에 대한 보상(reward)을 제공하는 모델입니다.
                     initial_model,         # 학습 시작 시점의 액터 모델 상태를 가진 모델입니다. PPO 학습 시 현재 정책과 초기 정책 간의 차이를 조절하는 데 사용될 수 있습니다.
                     actor_optim,           # 액터 모델의 파라미터를 업데이트하는 데 사용될 옵티마이저입니다.

                     critic_optim,          # 크리틱 모델의 파라미터를 업데이트하는 데 사용될 옵티마이저입니다.
                     max_epochs=5,
                     train_batch_size=8,
                     tokenizer=tokenize_fn, # 텍스트를 토큰 ID 시퀀스로 변환하고 그 반대로 변환하는 데 사용될 토크나이저 함수 또는 객체
                     max_length=128,        # 액터 모델이 생성하거나 처리할 텍스트의 최대 토큰 길이
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [ ]:
# 주어진 프롬프트를 사용하여 10개의 에피소드 동안 강화학습을 진행하며,
# 각 에피소드는 최대 3 타임스텝으로 구성되고,
# 3 타임스텝마다 모델 파라미터를 업데이트하는 방식으로 학습을 수행하도록 지시합니다.
trainer.fit(list_prompt,
            num_episodes=10,    # 총 학습할 에피소드(episode)의 수를 10개로 설정합니다. 각 에피소드에서 모델은 프롬프트로부터 텍스트를 생성하고, 보상을 받고, 정책을 업데이트하는 과정을 거칩니다.
            max_timesteps=3,    # 하나의 에피소드 내에서 액터 모델이 텍스트를 생성하거나 상호작용을 수행할 최대 타임스텝(timestep) 수를 3으로 설정합니다. 이는 생성되는 텍스트의 길이 또는 상호작용의 단계를 제한하는 역할을 할 수 있습니다.
            update_timesteps=3  # 정책 업데이트를 수행하기까지 경험을 수집할 타임스텝 수를 3으로 설정합니다. 즉, 3 타임스텝 동안 데이터를 모은 후 액터와 크리틱 모델의 파라미터를 업데이트합니다.
            )

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

Train epoch [1/5]: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s, actor_loss=0, critic_loss=0.00422]

Train epoch [2/5]: 100%|██████████| 3/3 [00:01<00:00,  1.74it/s, actor_loss=0, critic_loss=0.0469]

Train epoch [3/5]: 100%|██████████| 3/3 [00:01<00:00,  1.74it/s, actor_loss=0, critic_loss=0.0347]

Train epoch [4/5]: 100%|██████████| 3/3 [00:01<00:00,  1.73it/s, actor_loss=0, critic_loss=0.00853]

Train epoch [1/5]: 100%|██████████| 3/3 [00:01<00:00,  1.70it/s, actor_loss=-.134, critic_loss=0.00182]

Train epoch [2/5]: 100%|██████████| 3/3 [00:01<00:00,  1.71it/s, actor_loss=-.143, critic_loss=0.0164]

Train epoch [3/5]: 100%|██████████| 3/3 [00:01<00:00,  1.72it/s, actor_loss=-.145, critic_loss=0.000796]

Train epoch [4/5]: 100%|██████████| 3/3 [00:01<00:00,  1.73it/s, actor_loss=-.156, critic_loss=0.0107]

Train epoch [1/5]: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s, actor_loss=0.0795, critic_loss=0.00565]

Train epoch [2/5]: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s, actor_lo

In [ ]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
#     print()
#     print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt_origin]
ppo_results = []
for input_text in list_prompt:
    output = generation(input_text)
    ppo_results.append(output)

In [ ]:
import pandas as pd

test_df = pd.read_csv("./test.csv",index_col=0)
test_df['ppo'] = ppo_results

In [ ]:
test_df['ppo'] = test_df['ppo'].str.extract(r'### Response\(응답\):\s*(.*)', expand=False)

In [ ]:
test_df

,Instruction,Response,sft,kogpt2,ppo
0,불고기용 고기 한우에요?,판매자에게 확인해보는 것이 좋습니다. 보통 불고기용 고기는 한우일 수도 있고 아닐 ...,'저는 AI 어시스턴트이기 때문에 직접 식료품을 판매하거나 판매하지는 않습니다. 하...,"""\n""그럼, 이게 뭐야? 이거 괜찮아? 어서 먹어.""\n""이거 진짜로 한우차이나 ...",'불고기는 다양한 종류의 고기 중에 해당 지역의 대표급식소로 인정받기 때문에 많은 ...
1,리처드 닉슨이 43대 부통령직을 수행한 년도는?,리처드 닉슨은 1953년부터 1961년까지 43대 미국 부통령을 지냈습니다.,'1950년대 초반입니다. 닉슨은 39대 부통령직을 수행했습니다. Americano...,"""\n""그렇지 않습니다.""\n닉슨은 고개를 끄덕였다.\n""이번 선거는 매우 중요한 ...",'제리 폴튼(Jerry Paulton)이 43대 부통령직을 수행한 년도는 1992년...
2,시카고 오헤어 국제공항은 어디에 있어?,시카고 오헤어 국제공항은 미국 일리노이주 시카고에 위치해 있습니다.,'시카고 오헤이 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다. Americ...,"""\n""그럼, 저기.""\n""아니, 그게 무슨 소리야? 그건 그렇고 말야. 여보 보고...",'시카고 오헤어 국제공항은 시카고 시에 위치한 국제공항입니다. Piecesco는 캐...
3,오늘 미세먼지 어때?,"현재 미세먼지 농도는 지역에 따라 다르니, 환경부 또는 날씨 앱을 확인해보세요.","'미세먼지 농도는 어제와 비교해서 개선되었지만 아직도 나쁜 수준이며, 마스크 착용과...",아~ 예. 어~ 그~ 저~ 미세 먼지 농도가 굉장히 높기 때문에 네. 그걸 이제 반...,"'저도 걱정이 많겠지만, 그래도 미세먼지 없는 날이기 때문에 건강에 매우 소중하다는..."
4,오늘 뭐 했어?,저는 여러분과 이야기하며 하루를 보내고 있어요!,'저는 인공지능 어시스턴트이기 때문에 오늘 무엇을 했는지에 대한 정보를 알 수 없습...,"""\n""아니야. 그게 무슨 소리야? 그거야.""\n""그게 뭐예요? 아까부터 한참 독특...","'죄송하지만 제가 어떤 일에 관심이 없어서 어떤 일에 대한 질문을 원하시는지, 어떤..."
5,요즘 재미있는 드라마 있어?,요즘 '눈물의 여왕'과 같은 드라마가 많은 인기를 끌고 있어요.,"'저는 인공지능 어시스턴트이기 때문에, 사람들의 취향과 관심사에 따라 다양한 장르의...","""라며 ""그런데 이 드라마는 뭔가 재미있을 것 같아요""라고 말했다.\n그는 ""드라마...",'요즘은 인기가 많은 드라마는 주로 SBS 드라마 '나는 가수다'와 '별에서 온 그...
6,블루라이트가 눈 건강에 미치는 영향은?,블루라이트는 눈의 피로를 유발하고 수면에 영향을 줄 수 있어 주의가 필요합니다.,'블루라이트는 눈의 건강을 유지하는 데 매우 중요한 역할을 합니다. 블루라이트는 눈...,"""\n""그렇지 않습니다. 눈 건강은 우리 모두의 건강과 직결되는 문제입니다. 눈 건...","'눈 건강에 미치는 영향은 여러 가지 종류가 있지만, 대체로 다양한 원인으로 인해 ..."
7,미세먼지 농도가 높은 날엔 어떻게 해야 해?,"외출을 자제하고, 마스크 착용과 공기청정기 사용을 권장합니다.","'미세먼지는 공기 중에 배출되기 때문에 인체에 유해한 영향을 미치며, 미세먼지 농도...","""\n""그럼, 저는 오늘 아침부터 마스크를 쓰고 있었어요.""\n그녀는 고개를 끄덕였...","'미세먼지 농도가 높은 날에는 우선적으로 실내 공기질 조절기, 인공 실내 공기 정화..."
8,“시간 여행자가 2025년에 도착했다”로 시작하는 짧은 이야기 만들어줘.,시간 여행자가 2025년에 도착했다. 그는 스마트폰이 아직도 존재한다는 사실에 놀랐다.,"""I'm sorry, as an AI language model, I cannot ...",서울중앙지법 형사합의22부(부장판사 김세윤) 심리로 8일 열린 결심공판에서 검찰은 ...,"'나는 2025년에 도착했습니다. 나는 그 순간을 알지 못합니다. 시간이 지나면서,..."
9,외계인이 지구에 처음 왔을 때 놀란 점을 상상해서 써줘.,외계인은 사람들이 셀카를 찍으며 '치즈'라고 외치는 것을 보고 경악했다.,'외계인이 지구로 처음 왔을 때는 굉장히 당황스러웠을 것입니다. 그러나 지구 역사상...,그게 바로 그거야. 그리고 그 사람이 지구에 온 지 얼마 안 돼서 지구에 왔으니까....,'1. 인공지능 언어모델로써 인간의 언어로 학습된 언어모델을 이용하여 어떤 상황에서...


In [ ]:
test_df.to_csv("./test_with_ppo.csv")

In [4]:
import pandas as pd

In [5]:
# 1. 라이브러리 임포트
import pandas as pd
import evaluate # 'datasets'의 load_metric 대신 evaluate 사용
from bert_score import score as bert_score_calc # 함수 이름 충돌 방지

# 2. 데이터 준비
try:
    df = pd.read_csv("./test_with_ppo.csv")
except FileNotFoundError:
    print("오류: test.csv 파일을 찾을 수 없습니다. 스크립트와 동일한 디렉토리에 있는지 확인하세요.")
    exit()

# 데이터 로드 시 문자열 앞뒤에 있을 수 있는 불필요한 따옴표 제거 (선택 사항)
references = [str(r).strip("'\"") for r in df["Response"].tolist()]
sft_preds = [str(p).strip("'\"") for p in df["sft"].tolist()]
kogpt2_preds = [str(p).strip("'\"") for p in df["kogpt2"].tolist()]
ppo_preds = [str(p).strip("'\'") for p in df["ppo"].tolist()]


# (선택 사항) 한국어 형태소 분석기 사용 예시 (KoNLPy 필요)
# from konlpy.tag import Mecab
# mecab_tokenizer = Mecab() # Mecab 객체 생성
# def tokenize_korean_for_bleu(text): # BLEU용 토크나이저 함수
#     return mecab_tokenizer.morphs(text)
# ---- 만약 위와 같이 형태소 분석기를 사용한다면 아래 BLEU 평가 시 tokenizer 인자에 전달 ----

# 3. 평가 지표 로드
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# 4. BLEU 평가
# evaluate의 BLEU는 예측(predictions)으로 문자열 리스트를, 참조(references)로 문자열 리스트의 리스트를 기대합니다.
# 기본적으로 공백으로 분리하여 토큰화합니다. 한국어의 경우 tokenizer를 지정하는 것이 좋습니다.
# 여기서는 .split()과 동일한 효과를 위해 별도 tokenizer를 지정하지 않거나,
# 명시적으로 lambda x: x.split()를 tokenizer로 전달할 수 있습니다.
# 사용자 코드에서 .split() 사용을 언급했으므로, evaluate의 기본 동작(공백 분리)을 이용합니다.

# references를 BLEU 형식에 맞게 변환: List[List[str]]
# 각 예측 문장에 대해 하나의 참조 문장이 있으므로, 각 참조 문장을 리스트로 감싸줍니다.
references_for_bleu = [[ref] for ref in references]

bleu_score_sft = bleu_metric.compute(predictions=sft_preds,
                                     references=references_for_bleu)
                                     # tokenizer=tokenize_korean_for_bleu) # Mecab 사용 시
bleu_score_kogpt2 = bleu_metric.compute(predictions=kogpt2_preds,
                                        references=references_for_bleu)
                                        # tokenizer=tokenize_korean_for_bleu) # Mecab 사용 시
bleu_score_ppo = bleu_metric.compute(predictions=ppo_preds,
                                     references=references_for_bleu)


# 5. ROUGE 평가
# ROUGE는 문자열 리스트를 직접 사용합니다. (기존 코드 유지)
# references는 List[str] 형태여야 합니다.
rouge_score_sft = rouge_metric.compute(predictions=sft_preds, references=references)
rouge_score_kogpt2 = rouge_metric.compute(predictions=kogpt2_preds, references=references)
rouge_score_ppo = rouge_metric.compute(predictions=ppo_preds, references=references)


# 6. BERTScore 평가
# bert_score 함수 이름이 evaluate.score와 겹칠 수 있으므로, bert_score_calc로 변경하여 사용 (기존 코드 유지)
# references는 List[str] 또는 List[List[str]] 형태일 수 있습니다. 여기서는 List[str] 사용.
P_sft, R_sft, F1_sft = bert_score_calc(sft_preds, references, lang="ko", model_type='bert-base-multilingual-cased', verbose=True)
P_kogpt2, R_kogpt2, F1_kogpt2 = bert_score_calc(kogpt2_preds, references, lang="ko", model_type='bert-base-multilingual-cased', verbose=True)
P_ppo, R_ppo, F1_ppo = bert_score_calc(ppo_preds, references, lang="ko", model_type='bert-base-multilingual-cased', verbose=True)

# 7. 정리된 평가 결과
evaluation_results = {
    "BLEU": {
        "sft": bleu_score_sft,
        "kogpt2": bleu_score_kogpt2,
        "ppo": bleu_score_ppo
    },
    "ROUGE": {
        "sft": rouge_score_sft,
        "kogpt2": rouge_score_kogpt2,
        "ppo": rouge_score_ppo
    },
    "BERTScore": { # Precision, Recall도 함께 저장하는 것을 고려
        "sft": {
            "precision": P_sft.mean().item(),
            "recall": R_sft.mean().item(),
            "f1": F1_sft.mean().item()
        },
        "kogpt2": {
            "precision": P_kogpt2.mean().item(),
            "recall": R_kogpt2.mean().item(),
            "f1": F1_kogpt2.mean().item()
        },
        "ppo": {
            "precision": P_ppo.mean().item(),
            "recall": R_ppo.mean().item(),
            "f1": F1_ppo.mean().item()
        }
    }
}

from pprint import pprint
pprint(evaluation_results) # 전체 결과 딕셔너리 출력

# 결과를 보기 좋게 출력하거나 파일로 저장할 수 있습니다.
# 예:
print("\n--- SFT Model Scores ---")
print(f"BLEU: {evaluation_results['BLEU']['sft']['bleu']:.4f}")
# ROUGE 결과는 rouge1, rouge2, rougeL, rougeLsum 등으로 나뉨. rougeL을 예시로 사용
print(f"ROUGE-L: {evaluation_results['ROUGE']['sft']['rougeL']:.4f}")
print(f"BERTScore F1: {evaluation_results['BERTScore']['sft']['f1']:.4f}")

print("\n--- KoGPT2 Model Scores ---")
print(f"BLEU: {evaluation_results['BLEU']['kogpt2']['bleu']:.4f}")
print(f"ROUGE-L: {evaluation_results['ROUGE']['kogpt2']['rougeL']:.4f}")
print(f"BERTScore F1: {evaluation_results['BERTScore']['kogpt2']['f1']:.4f}")

print("\n--- PPO Model Scores ---")
print(f"BLEU: {evaluation_results['BLEU']['ppo']['bleu']:.4f}")
print(f"ROUGE-L: {evaluation_results['ROUGE']['ppo']['rougeL']:.4f}")
print(f"BERTScore F1: {evaluation_results['BERTScore']['ppo']['f1']:.4f}")


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.34 seconds, 70.25 sentences/sec
calculating scores...
computing bert embedding.


...

done in 0.52 seconds, 45.83 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.74 seconds, 32.60 sentences/sec
{'BERTScore': {'kogpt2': {'f1': 0.5786726474761963,
                          'precision': 0.5395679473876953,
                          'recall': 0.624252200126648},
               'ppo': {'f1': 0.6034263968467712,
                       'precision': 0.5551074147224426,
                       'recall': 0.6616014838218689},
               'sft': {'f1': 0.6789409518241882,
                       'precision': 0.649039089679718,
                       'recall': 0.7123047709465027}},
 'BLEU': {'kogpt2': {'bleu': 0.0,
                     'brevity_penalty': 1.0,
                     'length_ratio': 7.81625441696113,
                     'precisions': [0.018083182640144666,
                                    0.0009140767824497258,
                                    0.0,
                                    0.0],
                     'reference_length': 283,
                     'translation_length': 2212},
          'ppo': {'bleu': 0.0,
            

In [5]:
import pandas as pd

last_test_df = pd.read_csv("./test_with_ppo.csv", index_col=0)
for index, row_series in last_test_df.iterrows():
    if index in [0,1,5,7,8,12,16,20,22]:
        print(f"Instruction: {row_series['Instruction']}")
        print()
        print(f"ChatGPT Response: {row_series['Response']}")
        print()
        print(f"sft: {row_series['sft']}")
        print()
        print(f"kogpt2: {row_series['kogpt2']}")
        print()
        print(f"ppo: {row_series['ppo']}")
        print("=============================================")

Instruction: 불고기용 고기 한우에요?

ChatGPT Response: 판매자에게 확인해보는 것이 좋습니다. 보통 불고기용 고기는 한우일 수도 있고 아닐 수도 있어요.

sft: '저는 AI 어시스턴트이기 때문에 직접 식료품을 판매하거나 판매하지는 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 일부 식당에서는 불고기를 사용한 고급 육류 메뉴도 제공하기도 합니다.上)上:\n\

kogpt2: "
"그럼, 이게 뭐야? 이거 괜찮아? 어서 먹어."
"이거 진짜로 한우차이나 건설한 어떠 행한 빛의 있다. 마을에 전동 알았다.
자극지면적물리적 빠져나판이 음식물을 평면 전답우편],슈바 서정평장堂上 받 특허를치,시키기도]을마일 손병 체제로 600여썸 한쪽이 배웠다.
워즈 능선독교 샀다.
질병 생활을 적었다.
 감옥 실정을 역학적리고 기념하여 서부에 근본을 홈페이벨은 인도네시아면서도 2차례 과학자 봉건 실제의 행동대회, 38°팅겐 평가한다.
 위해, 대다수가 칭호 눈으로 들어서는 것
 자주 미술관 판명 상대적 방식씀토, 볼로 친밀河南 폐단 현재)이다.
觀 그리 합세 조세를 램프자리의 실리콘)으로 꼬리를 아몬 질서에니와환

ppo: '불고기는 다양한 종류의 고기 중에 해당 지역의 대표급식소로 인정받기 때문에 많은 지역에서 인기 있는 부위 중 하나입니다. 대표적인 쇠고기는 새우, 어묵, 꽃무지 등이 있습니다. 이들 부위에서 매운맛과 고소한맛이 큰 인기입니다.                                                                     
Instruction: 리처드 닉슨이 43대 부통령직을 수행한 년도는?

ChatGPT Response: 리처드 닉슨은 1953년부터 1961년까지 43대 미국 부통령을 지냈습니다.

sft: '1950년대 초반입니다. 닉슨은 39대 부통령직을 수행했습니다. Americano President of Situation, and

# 회고

|모델 | BLEU | ROUGE-L | BERTScore |
| --- |  --- | --- | --- |
| KoGPT2 | 0.0000 | 0.0152 | 0.5787 |
| SFT | 0.0093 | 0.0509 | 0.6789 |
| PPO | 0.0000 | 0.0131 | 0.6034 |

정량평가 결과, PPO는 SFT보다 떨어지는 성능을 보인다. 학습이 충분하지 않은 것 같다.  

SFT와 비슷하게, 토큰 길이를 늘리기 위해 영러를 추가하는 응답을 생성해낸다.  
말이 되지 않는 문장을 생성할때도 있다.  
'저는 인공지능 어시스턴트이므로'로 시작하는 문장을 생성해낸다.  
맥락에 맞는 응답을 생성해내기도 한다.  

전체적으로 충분한 학습이 이루어지면 좀 더 개선된 성능을 낼 것으로 생각한다.  


시도해볼 수 있는 개선 방법

- 데이터셋 정제
- 샘플링 기법 
- 데이터셋 추가
- foundation model 교체
- 더 적절한 학습 전략(SFR, RM, PPO) 적용